<a href="https://colab.research.google.com/github/ucfilho/Doubts_Students/blob/master/LSTM_shift_fixed_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [3]:
!git clone https://github.com/ucfilho/Machine_Learning_DataSet
%cd Machine_Learning_DataSet

fatal: destination path 'Machine_Learning_DataSet' already exists and is not an empty directory.
/content/Machine_Learning_DataSet


In [4]:
# df = pd.read_csv("Superstore.csv", encoding='windows-1252')
df = pd.read_csv("dados_precipitacao_teste.csv")
print(df.head(3)) # shows headers with top 3 rows

         data  Ent62_SSTtSSA  1_SF  2_SF  3_SF  4_SF      5_SF      7_SF  \
0  2022-11-03       7.454262   0.0   0.0   0.0   0.0  0.000000  0.266250   
1  2022-11-04       6.578652   0.0   0.0   0.0   0.0  0.000000  0.234375   
2  2022-11-05      12.633390   0.0   0.0   0.0   0.0  0.017083  0.254792   

       8_SF      9_SF  ...  Sai62_BalPop  Sai63_BalPop  EntCresc_BalPop  \
0  0.513333  0.807500  ...  7.257745e+14  8.463053e+14     1.572063e+15   
1  0.492917  0.791875  ...  5.409959e+14  7.163564e+14     1.257348e+15   
2  0.454167  0.677708  ...  7.171353e+14  7.372483e+14     1.454377e+15   

      SG_BalPop  SaiCresc_BalPop    End_BalPop  44_Hidrato  \
0  3.535153e+14     1.909924e+15  1.114866e+15    1.483333   
1  5.563443e+14     1.814094e+15  1.709214e+15    2.070000   
2  3.163980e+14     1.771134e+15  1.224233e+15    2.153333   

   DensAparente_Alumina  25_44_SF  25_44_SF_2  
0                 936.0  0.146731    0.021530  
1                 940.0  0.166737    0.027801  
2

In [5]:
time = df['data'].values
X = df.drop(['data','25_44_SF'],axis=1)
#X.index = time
y = df[['25_44_SF']]
nomes = ['25_44_SF']
y.index = time
P = 7 # numero de periodos para tras (trocar pelo desejado)
y_new = {}
y_new[nomes[0]]=y.values
for i in range(P):
  w = '25_44_SF' + str(i+1)
  nomes.append(w)
  y_new[w]= y.shift(i+1).values


In [6]:
dic_ ={}
for i in range(P):
  dic_[nomes[i]] = list(y_new[nomes[i]].reshape(-1))

y = pd.DataFrame(dic_)
y = np.array(y)

In [7]:
rows, cols = X.shape
ntrain = int(rows*0.8)
X_train=X.iloc[0:ntrain,:].copy()
y_train= y[0:ntrain,:].copy()
X_test=X.drop(X_train.index)
y_test=np.delete(y,range(ntrain))
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [8]:
rows,cols

(269, 240)

In [9]:
#, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
#X_train
scalerX = MinMaxScaler()
scalerY = MinMaxScaler()
scalerX.fit(X_train)
scalerY.fit(y_train)
X_train = scalerX.transform(X_train)
X_test = scalerX.transform(X_test)
y_train= scalerY.transform(y_train)
y_test= scalerY.transform(y_test)

In [11]:

for i in range(1):
  '''
  model = Sequential()
  model.add(LSTM(units=32, activation='tanh',input_shape=(X_train.shape[1], 4)))
  # units=32
  model.add(Dense(1))
  '''
  # model = keras.Sequential()
  model = Sequential()
  # Add an Embedding layer expecting input vocab of size 1000, and
  # output embedding dimension of size 64.
  #model.add(layers.Embedding(input_dim=1000, output_dim=64))

  # Add a LSTM layer with 128 internal units.
  model.add(LSTM(128))

  # Add a Dense layer with 10 units.
  model.add(Dense(P))

  #model.summary()
  model.compile(loss='mean_squared_error', optimizer='adam')
  X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
  X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
  # reshape input to be [samples, time steps, features]
  # reshape input to be [samples, time steps, features]
  #trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
  #testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
  model.fit(X_train, y_train, epochs=200, batch_size=1, verbose=0)
  y_calc_train=model.predict(X_train)
  y_calc_test=model.predict(X_test)
  mse_test=mean_squared_error(y_test,y_calc_test)
  R2_test=r2_score(y_test,y_calc_test)
  mse_train=mean_squared_error(y_train,y_calc_train)
  R2_train=r2_score(y_train,y_calc_train)

ValueError: ignored

In [ ]:
X_test.shape

In [ ]:
# expressa os dois dados em figura
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(y_test,y_calc_test, s=10, c='b', marker="s", label='teste')
ax1.scatter(y_train,y_calc_train, s=10, c='r', marker="o", label='treino')
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])
plt.legend(loc='upper left')
plt.ylabel('Y Calc')
plt.xlabel('Y Obs')
plt.show()

In [ ]:
mse=mean_squared_error(y_test,y_calc_test)
print("MSE teste=",mse)
R2=r2_score(y_test,y_calc_test)
print("R^2 teste=",R2)

In [ ]:
mse=mean_squared_error(y_train,y_calc_train)
print("MSE train=",mse)
R2=r2_score(y_train,y_calc_train)
print("R^2 train=",R2)

In [ ]:
y_calc = y_calc_train.tolist()

In [ ]:
for yi in y_calc_test:
  y_calc.append(yi)

In [ ]:
y_calc = np.array(y_calc).reshape(1,-1)
y_calc = scalerY.inverse_transform(y_calc)

In [ ]:
line1, =plt.plot(range(len(y)),y,c='r')
line2, =plt.plot(range(len(y)),y_calc[0], c='b')
plt.legend([line1,line2],['true','ANN'],loc='upper right')
plt.xlabel('Semanas')
plt.ylabel('Vendas em milhões/semana')
plt.show()

In [ ]:
df['Product ID'].shape

In [ ]:
y_calc

In [ ]:
y

In [ ]:
plt.plot(furniture)
plt.xticks(rotation=45)
plt.ylabel('Vendas em milhões')

In [ ]:
# save excel file
from google.colab import files
Arq = "serie_output.xlsx"
dataframe.to_excel(Arq)
files.download(Arq)

In [ ]:
sum(df['Profit'].values)